# All functions for personality prediction

## Prep

In [1]:
# Import packages

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.util import bigrams, ngrams

import re
import string
from string import punctuation

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

import gensim
from gensim import corpora, models

from empath import Empath

from collections import Counter
from num2words import num2words
from lexicalrichness import LexicalRichness
import textblob


import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
tqdm.pandas()
import time
import datetime
import random
random.seed(32)


[nltk_data] Downloading package punkt to /home/sophia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sophia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sophia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /home/sophia/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


## Prepare dataset

In [2]:
# # Import dataset with comments (big five labels)
# df = pd.read_csv('/home/sophia/ma_py/pandora_bigfive.csv')

# # Import augmented big five comment dataset
# df = pd.read_pickle("pandora_b5_newauthors_df.pkl")

# # Import dataset authors and delete not needed columns (big five labels)
# authors = pd.read_csv('/home/sophia/ma_py/author_profiles.csv')
# bigfive = authors[['author','agreeableness','openness','conscientiousness','extraversion','neuroticism']]
# bigfive = bigfive[bigfive['openness'].notna()]
# bigfive = bigfive[bigfive['conscientiousness'].notna()]
# bigfive = bigfive[bigfive['extraversion'].notna()]
# bigfive = bigfive[bigfive['agreeableness'].notna()]
# bigfive = bigfive[bigfive['neuroticism'].notna()]
# del authors

# Datasets with mbti und big five labels
df = pd.read_pickle("comments_uniondf.pkl")
bigfive = pd.read_pickle("uniondf.pkl")
bigfive

# remember to change name of output as well!!!!

,author,agreeableness,openness,conscientiousness,extraversion,neuroticism,mbti_ei,mbti_ns,mbti_tf,mbti_jp
0,-BlitzN9ne,50.0,85.0,15.0,50.0,30.0,1,1,1,0
1,-dyad-,60.0,67.0,45.0,10.0,47.0,0,1,0,0
2,12345jk12345,0.0,37.0,72.0,72.0,3.0,1,0,1,0
3,ACE_C0ND0R,10.0,80.0,74.0,27.0,18.0,0,1,1,1
4,A_Bra_and_a_Ham,1.0,50.0,61.0,90.0,3.0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
397,quakeroaks,3.0,85.0,19.0,50.0,92.0,1,1,0,0
398,rrgjl,0.0,92.0,52.0,1.0,79.0,0,1,1,1
399,seldomvanilla,21.0,96.0,86.0,32.0,0.0,0,1,1,1
400,turncloak471,6.0,91.0,88.0,15.0,4.0,0,1,1,1


In [3]:
traitlen = len(bigfive.columns.tolist())
# minus 1 because of author column
traitlen = traitlen-1 
traitlen

9

In [4]:
# Functions

# change language to numeric representation
# def numeric_lang(df):
#     # change lang to numerical representation
#     language = df['lang'].values.tolist()
#     language = set(language)
#     df['language']= np.select([df.lang == 'en', df.lang == 'es', df.lang == 'nl'], 
#                             [0, 1, 2], 
#                             default=3)
#     df = df.drop(columns=['lang'])

#     return df

# create time columns from UTC
def create_timecolumns(df):
    readable = []
    weekday = []
    month = []
    year = []
    hour = []
    for row in tqdm(df['created_utc']):
        item = datetime.datetime.fromtimestamp(row)
        weekday_item = item.strftime('%A')
        readable_item = datetime.datetime.fromtimestamp(row).isoformat()
        month.append(str(readable_item[5:7]))
        year.append(str(readable_item[0:4]))
        hour.append(str(readable_item[11:13]))
        readable.append(readable_item)
        weekday.append(weekday_item.lower())
    df['time'] = readable
    df['weekday'] = weekday
    df['month'] = month
    df['year'] = year
    df['daily'] = hour
    return df

# count occurences in time columns to get time distribution
def timecounter(lst, vocablst):
    if vocablst == 'weekday':
        vocab = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    elif vocablst == 'month':
        vocab = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    elif vocablst == 'hour':
        vocab = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', 
                 '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
    elif vocablst == 'year':
        vocab = ['2015', '2016', '2017', '2018', '2019']
    else:
        print("No valid input: vocab list")
    vectorizer = CountVectorizer(analyzer="word", vocabulary=vocab)
    vectors = vectorizer.fit_transform(lst)
    v = vectors.toarray()
    return v

def timeinterval(lst):
    max_lst = []
    median_lst = []
    mean_lst = []
    for sublst in lst:
        intlst = [int(item) for item in sublst]
        if len(intlst) > 1:
            intlst.sort()
            arr = np.array(intlst)
            diff_lst = np.diff(arr)
            max_lst.append(max(diff_lst))
            median_lst.append(np.median(diff_lst))
            mean_lst.append(np.mean(diff_lst))
        else:
            max_lst.append(-1)
            median_lst.append(-1)
            mean_lst.append(-1)
    return mean_lst, median_lst, max_lst

# create a list of all subreddits in the dataset
lst = df['subreddit'].tolist()
lst = [item.lower() for item in lst]
subredditset = set(lst)
subredditlist = list(subredditset)
subredditlength = len(subredditlist)

# count occurences of subreddits 
def subredditcounter(lst, subredditlst):
    vectorizer = CountVectorizer(analyzer="word", vocabulary=subredditlist)
    vectors = vectorizer.fit_transform(lst)
    v = vectors.toarray()
    return v

# aggregate dataset to get one row per author and create new columns for time and subreddit
def create_groupdf(df): 
#     print("\tCreate numeric language representation...")
#     df = numeric_lang(df)
    print("\tCreate time columns...")
    df = create_timecolumns(df)
    # create dictionary for aggregation function
    d = {'lang': ['nunique'], 'ratio_en': (lambda x : list(x)), 'controversiality': ['mean'], 'gilded': ['mean'], 'score':['mean'],
         'body': (' '. join), 'doc_body': (lambda x : list(x)),
         'utc': (lambda x : list(x)), 'subreddit': (' '. join), 'num_subreddit': ['nunique'],
         'weekday': (' '. join), 'month': (' '. join), 'year': (' '. join), 'daily': (' '. join)}
 
    # new ungrouped columns
    print("\tCreate new ungrouped columns...")
    df['body'] = df['body'].apply(lambda x: str(x))
    df['doc_body'] = df['body']
    df['num_subreddit'] = df['subreddit']
#     df['lang'] = df['language'].apply(lambda x: str(x))
    df['ratio_en'] = df['lang']
    df['utc'] = df['created_utc'].apply(lambda x: str(x))
#     df['subreddit'] = df['subreddit'].apply(lambda x: [x.lower()])
    df['subreddit'] = df['subreddit'].apply(lambda x: ''.join(x.lower()))
    counts = df['author'].value_counts()
    ndf = pd.DataFrame(counts)
    ndf.reset_index(inplace=True)
    ndf.rename(columns = {'index':'author', 'author': 'n_comments'}, inplace = True)
    
    # create df groupd by author + transform
    print("\tGroup df by author...")
    groupdf = df.groupby(['author']).agg(d)
    groupdf = groupdf.reset_index()
    groupdf.columns = groupdf.columns.droplevel(1)
    groupdf.merge(ndf, left_on='author', right_on='author')
    
    return groupdf
    
def create_new_columns(df):    
    # controversiality
    print("\tCreate controversiality column...")
    df['controversiality'] = df['controversiality'].fillna(0)
    # gilded
    print("\tCreate mean_gilded...")
    df['gilded'] = df['gilded'].fillna(0)
    # ratio of english comments
    newcolumn = []
    for row in df['ratio_en']:
        other = [value for value in row if value != 'en']
        english = row.count('en')
        if len(other) == 0:
            newcolumn.append(1)
        else: 
            newcolumn.append(english/len(other))
    df['ratio_en'] = newcolumn
    # number of comments per subreddit
    print("\tCreate subreddit_dist...")
    subreddit_predist = subredditcounter(df['subreddit'], subredditlist)
    subreddit_predist = subreddit_predist.tolist()
    df['subreddit_dist'] = subreddit_predist
    # entropy
    df['entropy'] = df['subreddit_dist'].apply(lambda x: entropy(x, base=2))
    # time
    print("\tCompute time intervals...")
    df['mean_time'], df['median_time'], df['max_time'] = timeinterval(df['utc'])
    print("\tCreate weekday_dist...")
    weekday = timecounter(df['weekday'], 'weekday')
    weekday = weekday.tolist()
    df['weekday_dist'] = weekday
    print("\tCreate month_dist...")
    month = timecounter(df['month'], 'month')
    month = month.tolist()
    df['month_dist'] = month
    print("\tCreate year_dist...")
    year = timecounter(df['year'], 'year')
    year = year.tolist()
    df['year_dist'] = year
    print("\tCreate day_dist...")
    day = timecounter(df['daily'], 'hour')
    day = day.tolist()
    df['daily_dist'] = day
    
    print("\tCreate new aggregated df...")
    newdf = df[['author', 'body', 'doc_body', 'utc', 'score', 'controversiality', 
                'gilded', 'ratio_en', 'num_subreddit', 'subreddit_dist', 'entropy', 'mean_time', 'median_time', 'max_time', 'weekday_dist', 
                'month_dist', 'year_dist', 'daily_dist', 'lang']]
    print("\tSort new aggregated df...")
    newdf = newdf.sort_values(by='author')
    print("\tDrop duplicates in new aggregated df...")
    newdf = newdf.drop_duplicates(subset=['author'])
    return newdf

# get one column for each feature in the distributions of time and subreddit
weekday = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
month = ['january', 'february', 'march', 'april', 'may', 'june', 'juli', 'august', 'september', 'october', 'november', 'december']
year = ['2015', '2016', '2017', '2018', '2019']
hour = ['hour01', 'hour02', 'hour03', 'hour04', 'hour05', 'hour06', 'hour07', 'hour08', 'hour09', 'hour10', 
        'hour11', 'hour12', 'hour13', 'hour14', 'hour15', 'hour16', 'hour17', 'hour18', 'hour19', 'hour20', 
        'hour21', 'hour22', 'hour23', 'hour24']
timelen = len(weekday+month+year+hour)

def onecolumnperdatapoint(df, column, namelist):
    for i in tqdm(range(len(namelist))):
        df[namelist[i]] = df[column].apply(lambda x:[x[i]])
        df[namelist[i]] = [item[0] for item in df[namelist[i]]]
    return df

In [5]:
# Wrapper for commentdf
def create_commentdf(df):
    print("Create new df grouped by author...")
    groupdf = create_groupdf(df)
    print("Create new columns with features...")
    pandora = create_new_columns(groupdf)
    print("Distribute the weekday_dist to several columns...")
    pandora = onecolumnperdatapoint(pandora, 'weekday_dist', weekday)
    print("Distribute the month_dist to several columns")
    pandora = onecolumnperdatapoint(pandora, 'month_dist', month)
    print("Distribute the year_dist to several columns...")
    pandora = onecolumnperdatapoint(pandora, 'year_dist', year)
    print("Distribute the daily_dist to several columns...")
    pandora = onecolumnperdatapoint(pandora, 'daily_dist', hour)
    print("Distribute the subreddit_dist to several columns...")
    pandora = onecolumnperdatapoint(pandora, 'subreddit_dist', subredditlist)
    print("Drop dist columns...")
    pandora.drop(['weekday_dist', 'month_dist', 'year_dist', 'daily_dist', 'subreddit_dist'], axis=1, inplace=True)
    return pandora

## Preprocessing

In [6]:
# Functions
# create binary representation of personality traits
def bigfive_cat(df):
    # change big five to binary representation
    df['trait', 'big5_a'] = df['trait', 'agreeableness'].apply(lambda x: 0 if x<50 else 1)
    df['trait', 'big5_o'] = df['trait', 'openness'].apply(lambda x: 0 if x<50 else 1)
    df['trait', 'big5_c'] = df['trait', 'conscientiousness'].apply(lambda x: 0 if x<50 else 1)
    df['trait', 'big5_e'] = df['trait', 'extraversion'].apply(lambda x: 0 if x<50 else 1)
    df['trait', 'big5_n'] = df['trait', 'neuroticism'].apply(lambda x: 0 if x<50 else 1)
    df['trait', 'big5_a_multi'] = df['trait', 'agreeableness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
    df['trait', 'big5_o_multi'] = df['trait', 'openness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
    df['trait', 'big5_c_multi'] = df['trait', 'conscientiousness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
    df['trait', 'big5_e_multi'] = df['trait', 'extraversion'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
    df['trait', 'big5_n_multi'] = df['trait', 'neuroticism'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
    return df

# define stopwordlist to use
def choose_stopwordlist(df, mode):
    if mode == 'NLTK':
        stopwordList = stopwords.words('english')
    if mode == 'NLTK-neg':
        stopwordList = stopwords.words('english')
        stopwordList.remove('no')
        stopwordList.remove('nor')
        stopwordList.remove('not')
    return stopwordList

# remove decontractions
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# create sentence tokens
def senttokenize(df):
    sentbody = []
    for row in tqdm(df['text', 'doc_body']):
        sentitem = []
        for item in row:
            sentences = sent_tokenize(item)
            sentitem.append(sentences)
        sentbody.append(sentitem)
    df['text', 'senttokens'] = sentbody
    return df

# lower words and remove special characters
def lower_special(df):
    newrow = []
    for row in tqdm(df['text', 'decon_body']):
        newcomment = []
        for comment in row:
            text_pre = ""
            for character in comment:
                if character.isalnum() or character.isspace():
                    character = character.lower()
                    text_pre += character
                else:
                    text_pre += " "
            newcomment.append(text_pre)
        newrow.append(newcomment)   
    df['text', 'probody'] = newrow
    return df

# remove stopwords
def remove_stopwords(df, stopwordList):
    newprobody = []
    for row in tqdm(df['text', 'probody']):
        newrowprobody = []
        for comment in row:
            words = [word for word in comment.split() if (word not in stopwordList)]
            newcomment = ' '.join(words)
            newrowprobody.append(newcomment)
        newprobody.append(newrowprobody)
    df['text', 'probody'] = newprobody
    return df

# change numbers to words and tokenize words

import decimal
def num_tokenize(df):    
    newbody_complete = []
    newprobody_complete = []
    # num2words
    for row in tqdm(df['text', 'probody']):
        newbody = []
        newprobody = []
        for sentence in row:
            # string to list
            inputtext = sentence.split()
            numlist = []
            for i in range(len(inputtext)):
                if inputtext[i].isnumeric():
                    numlist.append(i)
            for number in numlist:
                # deleted: fractions, superscripts, extremely large numbers, 卌卌, 一
                try:
                    inputtext[number] = num2words(inputtext[number])
                except decimal.InvalidOperation:
                    inputtext[number] = " "
                except OverflowError:
                    inputtext[number] = " "

            # list to string
            inputtext = [word for word in inputtext if word.isalpha()]
            celltext = ' '.join(inputtext)
            newprobody.append(celltext)
            # tokenize
            words = word_tokenize(celltext)
            newbody.append(words)
        newbody_complete.append(newbody)
        newprobody_complete.append(newprobody)
    df['text', 'probody'] = newprobody_complete
    df['text', 'tokens'] = newbody_complete
    return df

# Porter Stemmer
def stemming(df):
    ps = PorterStemmer()
    newtokens = []
    for row in tqdm(df['text', 'tokens']):
        newcomment = []
        for comment in row:
            words = [ps.stem(word) for word in comment]
            newcomment.append(words)
        newtokens.append(newcomment)
    df['text', 'tokens'] = newtokens
    return df

# bring columns of dataframe in correct order
def ordering(df):
    cols_tomove = ['trait', 'text', 'data', 'post', 'subtf']
#     cols_tomove = ['body', 'doc_body', 'decon_body', 'probody', 'tokens', 'senttokens', 'agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism', 'agree', 'openn', 'consc', 'extra', 'neuro']
    orderdf  = df[cols_tomove + [col for col in df.columns if col not in cols_tomove]]
    return orderdf

In [7]:
# Wrapper

def preprocess(df):
    # adjust some column representations
    df = bigfive_cat(df)
    # choose stopwordlist with or without negation
    stopwordList = choose_stopwordlist(df, mode='NLTK-neg')
    # decontract abbreviations (e.g., n't to not)
    print("Decontract...")
    df['text', 'decon_body'] = df['text', 'doc_body'].apply(lambda x:([decontracted(x) for x in x]))
    # create sentence tokens
    print("Tokenize Sentences...")
    df = senttokenize(df)
    # lower, remove stopwords, num2words, tokenize
    print("Lower words and remove special characters...")
    df = lower_special(df)
    print("Remove stopwords...")
    df = remove_stopwords(df, stopwordList)
    print("Change numbers to words and tokenize words...")
    df = num_tokenize(df)
    # porters stemmer
    print("Porters Stemmer...")
    df = stemming(df)
#     print("Order df...")
#     df = ordering(df)
    print("Done!")
    return df

## Extract features

In [8]:
# User features

# Preprocessing for LDA
def preprocess_lda(df):
    neglst = ["no", "not", "none", "nobody", "nothing", "neither", "nowhere", "never", "nay"]
    inputlst = []
    for row in tqdm(df['text', 'tokens']):
        rowlst = []
        for comment in row:
            rowlst.append([word for word in comment if (word not in neglst)])
        inputlst.append(rowlst)
    return inputlst


def apply_lda(df, inputlst, number, name):
    print("Start LDA...")
    lst = []
    for row in tqdm(inputlst):
        if len(row) < 2:
            lst.append(-1)
            print("\t-1 appended...")
        else:
            dictionary = corpora.Dictionary(row)
            corpus = [dictionary.doc2bow(text) for text in row]
            ldamodel = gensim.models.LdaMulticore(corpus, num_topics=number, id2word = dictionary, chunksize=160, workers=14)
            result = ldamodel.print_topics(num_topics=1, num_words=1)
            res = list(result)
            topic = [item[0] for item in res]
            lst.append(topic[0])
    df['lda', name] = lst
    return df

# Wrapper
def extract_userfeatures(df):
    print("Preprocessing for LDA...")
    inputlst = preprocess_lda(df)
    print("LDA with fifty topics: ")
    df = apply_lda(df, inputlst, 50, "ldafifty")
    print("LDA with onehundred topics: ")
    df = apply_lda(df, inputlst, 100, "ldahundred")
    return df

In [9]:
# Linguistic features (functions)

# other features that are not mentioned in the paper
def create_features(df):
    # Total number of characters (including space)
    print("\tCharacter count per author...")
    df['x_feat', 'char_count'] = df['text', 'body'].str.len()
    # Total number of stopwords
    print("\tNumber of stopwords per author...")
    stopwordList = stopwords.words('english')
    df['x_feat', 'stopwords'] = df['text', 'body'].apply(lambda x: len([x for x in x.split() if x in stopwordList]))
    # Total number of punctuation or special characters
    print("\tTotal number of punctuation per author...")
    df['x_feat', 'total_punc'] = df['text', 'body'].apply(lambda x: len([x for x in x.split() for j in x if j in string.punctuation]))
    # Total number of numerics
    print("\tTotal number of numerics per author...")
    df['x_feat', 'total_num'] = df['text', 'body'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
    # Total number of uppercase words
    print("\tTotal number of upper case words per author...")
    df['x_feat', 'total_uppercase'] = df['text', 'body'].apply(lambda x: len([x for x in x.split() if x.isupper()]))    
    return df

# type token ratio
def typetokenratio(df):
    ratiolst = []
    for comment in tqdm(df['text', 'body']):
            lex = LexicalRichness(comment)
            if lex.words == 0:
                ratiolst.append(0)
            else:
                ratio = lex.ttr
                ratiolst.append(ratio)
    df['lin_feat', 'ttr'] = ratiolst
    return df

# words per sentence
def wordcounter(df):
    lengthscore = []
    for row in tqdm(df['text', 'senttokens']):
        rowscore = []
        for comment in row:
            sentencescore = 0
            for senttoken in comment:
                length = len(senttoken.split())
                sentencescore += length
            if len(comment) > 1:
                sentencescore = sentencescore/len(comment)
        lengthscore.append(sentencescore)
        arr = np.array(lengthscore)
    df['lin_feat', 'words_per_sent'] = lengthscore
    return df

# words longer than six characters
def charcounter(df):
    charscore = []
    for row in tqdm(df['text', 'tokens']):
        for comment in row:
            rowcharscore = 0
            lencomment = len(comment)
            if lencomment == 0:
                score = 0
            else:
                number = 0
                for token in comment:
                    length = len(token)
                    if length > 5:
                        number+=1
                score = number/lencomment
            rowcharscore += score
        rowcharscore = rowcharscore/len(row)
        charscore.append(rowcharscore)
    df['lin_feat', 'wordslongersix'] = charscore
    return df

# POS tagger
def tagging(df):
    past = [] #VPA
    presence = [] #VPR
    adverbs = [] #RB
    prepositions = [] #PREP
    pronouns = [] #PR
    for comment in tqdm(df['text', 'body']):
            text = comment.split()
            tags = nltk.pos_tag(text)
            counts = Counter(tag for word,tag in tags)
            total = sum(counts.values())
            pron = counts['PRP'] + counts['PRP$']
            verbspr = counts['VB'] + counts['VBG'] + counts['VBP'] + counts['VBZ'] + counts['MD']
            verbspa = counts['VBD'] + counts['VBN']
            preps = counts['IN'] + counts['TO']
            counts['PR'] = pron
            counts['PREP'] = preps
            counts['VPR'] = verbspr #present tense
            counts['VPA'] = verbspa #past tense
            if total == 0:
                allcounts = dict((word, float(count)/1) for word,count in counts.items())
            else:
                allcounts = dict((word, float(count)/total) for word,count in counts.items())
            try:
                past.append(allcounts['VPA'])
            except KeyError:
                past.append(0)
            try:
                presence.append(allcounts['VPR'])
            except KeyError:
                presence.append(0)
            try:
                adverbs.append(allcounts['RB'])
            except KeyError:
                adverbs.append(0)
            try:
                prepositions.append(allcounts['PREP'])
            except KeyError:
                prepositions.append(0)
            try:
                pronouns.append(allcounts['PR'])
            except KeyError:
                pronouns.append(0)
    df['lin_feat', 'pasttense'] = past
    df['lin_feat', 'presencetense'] = presence
    df['lin_feat', 'adverbs'] = adverbs
    df['lin_feat', 'prepositions'] = prepositions
    df['lin_feat', 'pronouns'] = pronouns
    return df

def ngram_preprocessing(df):
    # convert input from list to string
    ngrams = []
    inputtext = []
    valid = True
    notvalid_lst =[]
    for row in tqdm(df['text', 'tokens']):
        valid_string = ""
        for comment in row:
            valid = True
            text = ' '.join(comment)
            i=0
            for char in text:
                if not(char in string.printable):
                    valid = False
                    notvalid_lst += [char]
                    i+=1
            if valid == True:
                textspace = text + " "
                valid_string += textspace
#         print(valid_string, "\n")
        inputtext.append(valid_string)
    if len(notvalid_lst) > 0:
        print("\nNumber of dismissed comments: ", i)
    print("Length of inputtext: ", len(inputtext))
    return inputtext
    
def ngrams(df, inputtext, n_min, n_max, ngramtype):
    vectorizer = TfidfVectorizer(ngram_range=(n_min,n_max), analyzer=ngramtype, max_features=(n_max-(n_min-1))*1000)
    print("Vectorize...")
    vectors = vectorizer.fit_transform(tqdm(inputtext))
    dense = vectors.todense()
    denselist = dense.tolist()
    print("Get feature names...")
    names = vectorizer.get_feature_names()
    print("Length of feature names: ", len(names))
    print("Create df...")
    ngramdf = pd.DataFrame(denselist, columns=names)
    return ngramdf

def merge_dfs(df1, df2, df3):
    ngramsdf = df1.join(df2, rsuffix="_char")
    ngramsdf['author_index'] = df3.index
    ngramsdf = ngramsdf.set_index('author_index')
    headers = (len(df1.columns))*['wordngram'] + (len(df2.columns))*['charngram']
    columns = ngramsdf.columns.values
    print("Headers: ", len(headers))
    print("Columns: ", len(columns))
    arrays = [headers] + [columns]
    ngramsdf.columns = pd.MultiIndex.from_arrays(arrays)
    newdf = df3.join(ngramsdf, rsuffix="_ngram")
    return newdf


In [10]:
# Wrapper for linguistic features

def extract_lin_features(df):
    print("Create additional features...")
    df = create_features(df)
    print("Create ttr...")
    df = typetokenratio(df)
    print("Count words per sentence...")
    df = wordcounter(df)
    print("Count words with more than six letters...")
    df = charcounter(df)
    print("POS-Tagger...")
    df = tagging(df)
    print("number of rows df", len(df))
#     df.to_pickle("union_linfeat_df_nongrams_nolda_allcomments.pkl")

    print("Ngrams...")
    print("Preprocessing for ngrams: ")
    inputtext = ngram_preprocessing(df)
    print("Create word ngrams...")
    wordngramsdf = ngrams(df, inputtext, 1, 3, "word")
    print("Create char ngrams...")
    charngramsdf = ngrams(df, inputtext, 2, 3, "char")
    print("Merge df...")
    gramsdf = merge_dfs(wordngramsdf, charngramsdf, df)
    return gramsdf


In [11]:
# Wordlists (functions)

# Empath
# create new categories with empath
def new_cat():
    empath = Empath()
    social = empath.create_category("social",["mate","talk","they"])
    humans = empath.create_category("humans",["adult","baby","boy"])
    cognitive = empath.create_category("cognitive",["cause","know","ought"])
    insight = empath.create_category("insight",["think","know","consider"])
    causation = empath.create_category("causation",["because","effect","hence"])
    discrepancy = empath.create_category("discrepancy",["should","would","could"])
    tentative = empath.create_category("tentative",["maybe","perhaps","guess"])
    certainty = empath.create_category("certainty",["always","never", "proof"])
    inhibition = empath.create_category("inhibition",["block","constrain","stop"])
    inclusive = empath.create_category("inclusive",["and","with","include"])
    exclusive = empath.create_category("exclusive",["but","without","exclude"])
    perceptual = empath.create_category("perceptual",["observing","hear","feeling"])
    see = empath.create_category("see",["view","saw","seen"])
    feel = empath.create_category("feel",["feels","touch","feeling"])
    biological = empath.create_category("biological",["eat","blood","pain"])
    relativity = empath.create_category("relativity",["area","bend","go"])
    motion = empath.create_category("motion",["arrive","car","go", "walk", "fly", "move", "run", "leave"])
    space = empath.create_category("space",["down","in","thin"])
    time = empath.create_category("time",["end","until","season"])
    agreement = empath.create_category("agreement", ["agree", "ok", "yes"])
    fillers = empath.create_category("fillers", ["like", "Imean", "yaknow"])
    nonfluencies = empath.create_category("nonfluencies", ["umm", "hm", "er"])
    conjunctions = empath.create_category("conjunctions", ["and", "but", "whereas"])
    quantifiers = empath.create_category("quantifiers", ["few", "many", "much"])
    numbers = empath.create_category("numbers", ["two", "fourteen", "thousand"])

def apply_empath(df):
    empath = Empath()
    print("Create new empath categories...")
    new_cat()
    print("Apply empath...")
    empathvalues = []
    empathcategories = ["swearing_terms", "social", "family", "friends", "humans", "emotional", "positive_emotion", 
                        "negative_emotion", "fear", "anger", "sadness", "cognitive", "insight", "causation", 
                        "discrepancy", "tentative", "certainty", "inhibition", "inclusive", "exclusive", 
                        "perceptual", "see", "hear", "feel", "biological", "body", "health", "sexual", "eat", 
                        "relativity", "space", "time", "work", "achievement", "leisure", "home", "money", 
                        "religion", "death" ,"agreement", "fillers", "nonfluencies", "conjunctions", "quantifiers", 
                        "numbers"]
    for sentence in tqdm(df['text', 'decon_body']):
        empathvalues.append(empath.analyze(sentence, categories=empathcategories, normalize=True))
    empathdf = pd.DataFrame(empathvalues)
    empathdf['author'] = df.index
    empathdf = empathdf.set_index('author')
    headers = 40*['empath'] + 5*['lin_feat']
    columns = empathdf.columns.values
    print(len(headers))
    print(len(columns))
    arrays = [headers] + [columns]
    empathdf.columns=pd.MultiIndex.from_arrays(arrays)
    newdf = df.join(empathdf, rsuffix="_empath")
    return newdf

In [12]:
# Import data for other wordlists
concretenessdf = pd.read_csv('/home/sophia/ma_py/psych_lists/concreteness.csv')
cdf = concretenessdf[['Conc.M']]
cmatrix = cdf.to_numpy()
concrete = concretenessdf['Word'].values.tolist()
del concretenessdf

happinessdf = pd.read_csv('/home/sophia/ma_py/psych_lists/happiness_ratings.csv')
hdf = happinessdf[['happiness_average']]
hmatrix = hdf.to_numpy()
happiness = happinessdf['word'].values.tolist()
del happinessdf

cursedf = pd.read_csv('/home/sophia/ma_py/psych_lists/mean_good_curse.csv')
cudf = cursedf[['mean_good_curse']]
cumatrix = cudf.to_numpy()
curse = cursedf['word'].values.tolist()
del cursedf

sensorydf = pd.read_csv('/home/sophia/ma_py/psych_lists/sensory_experience_ratings.csv')
serdf = sensorydf[['Average SER']]
sermatrix = serdf.to_numpy()
ser = sensorydf['Word'].values.tolist()
del sensorydf

alldf = pd.read_csv('/home/sophia/ma_py/psych_lists/sensory_ratings_all.csv')
newalldf = alldf[['Emotion', 'Polarity', 'Social', 'Moral', 'MotionSelf', 'Thought', 'Color', 'TasteSmell', 'Tactile', 'VisualForm', 'Auditory', 'Space', 'Quantity', 'Time', 'CNC', 'IMG', 'FAM']]
newalldf = newalldf.fillna(0)
allmatrix = newalldf.to_numpy()
allsens = alldf['Word'].values.tolist()
del alldf

valarodomdf = pd.read_csv('/home/sophia/ma_py/psych_lists/valence_arousal_dominence.csv')
vaddf = valarodomdf[['V.Mean.Sum', 'A.Mean.Sum', 'D.Mean.Sum']]
vadmatrix = vaddf.to_numpy()
vad = valarodomdf['Word'].values.tolist()
del valarodomdf

mrcdf = pd.read_csv('/home/sophia/ma_py/psych_lists/mrclists_c_p.csv', sep='\t', names=['word', 'cmean', 'pmean'])
cpdf = mrcdf[['cmean', 'pmean']]
cpmatrix = cpdf.to_numpy()
mrc = mrcdf['word'].values.tolist()
del mrcdf

# function for other wordlists

def preprocess_counting(df):
    inputtext = []
    for row in tqdm(df['text', 'decon_body']):
        text = ' '.join(row)
        inputtext.append(text) 
    return inputtext

def counter(inputtext, vocab):  
    vectorizer = CountVectorizer(analyzer="word", ngram_range=(1,1), vocabulary = vocab)
    print("\tVectorize...")
    vectors = vectorizer.fit_transform(tqdm(inputtext))
    v = vectors.toarray()
    return v

def multiply(matrix, ratings):
    # matrix multiplication 
    result = np.matmul(matrix, ratings)
    # divide each score with the number of words in the list to normalize
    if len(ratings) > 0:
        result = result/(len(ratings))
    return result

def aggregator(df, inputtext, vocab, ratings, name):
    print("\tCount...")
    count = counter(inputtext, vocab)
    print("\tMultiply...")
    result = multiply(count, ratings)
    num_rows, num_cols = result.shape
    
    if num_cols ==1:
        df['psych', name] = result
        return df
    else:
        resultdf = pd.DataFrame(result, columns=name)
        resultdf['author'] = df.index
        resultdf = resultdf.set_index('author')
        headers = (len(name))*['psych']
        columns = resultdf.columns.values
        print(len(headers))
        print(len(columns))
        arrays = [headers] + [columns]
        resultdf.columns=pd.MultiIndex.from_arrays(arrays)
        df = df.join(resultdf, rsuffix="_wordlist")
        return df   

In [13]:
# wordlists created manually
negations = ["no", "not", "none", "nobody", "nothing", "neither", "nowhere", "never", "nay"]
articles = ["a", "an", "the"]
future = ["will", "gonna"]
pers_pronouns = ["i", "me", "my", "mine", "myself", "you", "your", "yours", "yourself", "he", "him", "his",
                 "himself", "she", "her", "hers", "herself", "it", "its", "itself", "themself", "we", "us",
                 "our", "ours", "ourselves", "they", "them", "their", "theirs", "themselves"]
fp_sing = ["i", "me", "my", "mine", "myself"]
fp_plural = ["we", "us", "our", "ours", "ourselves"]
secondp = ["you", "your", "yours", "yourself"]
tp_sing = ["he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "themself"]
tp_plural = ["they", "them", "their", "theirs", "themselves"]
indef_pronouns = ["another", "anybody", "anyone", "anything", "each", "either", "enough", "everybody", 
                        "everyone", "everything", "little", "much", "neither", "nobody", "no one", "nothing", 
                        "one", "other", "somebody", "someone", "something", "both", "few", "fewer", "many", 
                        "others", "several", "all", "any", "more", "most", "none", "some", "such"]
aux_verbs = ["be", "am", "are", "is", "was", "were", "being", "can", "could", "do", "did", "does", "doing", 
             "have", "had", "has", "having", "may", "might", "must", "shall", "should", "will", "would"]

def list_counter(df, inputtext, vocab, name):
    total = []
    for row in tqdm(df['text', 'decon_body']):
        total.append(len(row))
    vectorizer = CountVectorizer(analyzer="word", ngram_range=(1,1), vocabulary = vocab)
    print("\tVectorize...")
    vectors = vectorizer.fit_transform(tqdm(inputtext))
    v = vectors.toarray()
    averagev = v.sum(axis=1)
    totalvector =  np.array(total)
    score = np.divide(averagev, totalvector)
    df['lin_feat', name] = score
    return df

In [14]:
# Wrapper for wordlists
def extract_wordlist_features(df):
    print("Empath...")
    empdf = apply_empath(df)
    # create scores for each word list and add them to df
    print("Preprocessing for wordlists...")
    inputtext = preprocess_counting(df)
    print("\nWordlist Concreteness: \n")
    psychdf = aggregator(empdf, inputtext, concrete, cmatrix, "concreteness")
    print("\nWordlist Happiness: \n")
    psychdf = aggregator(psychdf, inputtext, happiness, hmatrix, "happiness")
    print("\nWordlist Good_Curse: \n")
    psychdf = aggregator(psychdf, inputtext, curse, cumatrix, "good_curse")
    print("\n17 further wordlists: \n")
    psychdf = aggregator(psychdf, inputtext, allsens, allmatrix, ['emotion', 'polarity', 'social', 'moral', 'motionself', 'thought', 'color', 'tastesmell', 'tactile', 'visualform', 'auditory', 'space', 'quantity', 'time', 'CNC', 'IMG', 'FAM'])
    print("\nWordlist SER: \n")
    psychdf = aggregator(psychdf, inputtext, ser, sermatrix, "SER")
    print("\nWordlists Valence, Arousal, Dominance: \n")
    psychdf = aggregator(psychdf, inputtext, vad, vadmatrix, ['valence', 'arousal', 'dominance'])
    print("\nWordlist Negation: \n")
    psychdf = list_counter(psychdf, inputtext, negations, "negations")
    print("\nWordlist Articles: \n")
    psychdf = list_counter(psychdf, inputtext, articles, "articles")
    print("\nWordlist Future: \n")
    psychdf = list_counter(psychdf, inputtext, future, "future")
    print("\nWordlist personal pronouns: \n")
    psychdf = list_counter(psychdf, inputtext, pers_pronouns, "pers_pronouns")
    print("\nWordlist first person singular pronouns: \n")
    psychdf = list_counter(psychdf, inputtext, fp_sing, "fp_sing")
    print("\nWordlist first person plural pronouns: \n")
    psychdf = list_counter(psychdf, inputtext, fp_plural, "fp_plural")
    print("\nWordlist second person pronouns: \n")
    psychdf = list_counter(psychdf, inputtext, secondp, "secondp")
    print("\nWordlist third person singular pronouns: \n")
    psychdf = list_counter(psychdf, inputtext, tp_sing, "tp_sing")
    print("\nWordlist third person plural pronouns: \n")
    psychdf = list_counter(psychdf, inputtext, tp_plural, "tp_plural")
    print("\nWordlist indefinite pronouns: \n")
    psychdf = list_counter(psychdf, inputtext, indef_pronouns, "indef_pronouns")
    print("\nWordlist auxiliary verbs: \n")
    psychdf = list_counter(psychdf, inputtext, aux_verbs, "aux_verbs")
    print("\nWordlists from MRC (2): \n")
    psychdf = aggregator(psychdf, inputtext, mrc, cpmatrix, ["mrc_cmean", "mrc_pmean"])
    
    return psychdf

In [15]:
def all_in_one(df, bigfive):
    start = datetime.datetime.now()
    print("Start time:", str(start))
    print("Create comment df (name: pandora)...")
    pandora = create_commentdf(df)
    # merge commentdf and authordf
    print("Sort pandora df...")
    pandora= pandora.sort_values(by='author')
    print("Sort big five df...")
    bigfive= bigfive.sort_values(by='author')
    if pandora.index.name != 'author':
        print("Set pandora index...")
        pandora = pandora.set_index('author')
    if bigfive.index.name != 'author':
        print("Set bigfive index...")
        bigfive = bigfive.set_index('author')
    print("Join commentdf and authordf")
    pandoradf = pandora.join(bigfive)
    del pandora
    del bigfive

    # create multiindex
    print("Create multiindex...\n")
    headers = 2*['text'] + 1*['data'] + 4*['post'] + 5*['subtf'] + 1*['post']
    headers = headers + (timelen + subredditlength -1)*['subtf'] + traitlen*['trait']
    
    # check multiindex
    columns = pandoradf.columns.values
    predictorsfile=open('columns.txt','w')
    for index in range(len(columns)):
        predictorsfile.write(columns[index])
        predictorsfile.write('\n')
    predictorsfile.close()
    
    print("Length headers", len(headers))
    print("Length columns", len(columns))
    arrays = [headers] + [columns]
    pandoradf.columns=pd.MultiIndex.from_arrays(arrays)
    pandoradf.info(verbose=True)
    
    # preprocessing
    print("\n\nPreprocessing...\n")
    predf = preprocess(pandoradf)
#     predf.to_pickle("union_preprocess_df_allcomments.pkl")
    del pandoradf
    
    # create linguistic features
    print("\n\nCreate linguistic features...\n")
    lin_ngrams_df = extract_lin_features(predf)
#     lin_ngrams_df.to_pickle("union_lin_feat_df_allcomments.pkl")
    del predf
    
    # create features with empath and wordlists
    print("\n\nCreate features with wordlists...\n")
    psychdf = extract_wordlist_features(lin_ngrams_df)
#     psychdf.to_pickle("union_wordlists_lin_feat_df_allcomments.pkl")
    del lin_ngrams_df
        
    # create user features
    print("\n\nCreate user features (LDA)...\n")
    user_feat_df = extract_userfeatures(psychdf)
    
    end = datetime.datetime.now()
    print("End time:", str(end))
    
    return user_feat_df
    
featuredf = all_in_one(df, bigfive)
featuredf.to_pickle("b5mbtifeat.pkl")
print(featuredf.info())
featuredf.head()

Start time: 2021-04-09 11:27:44.480552
Create comment df (name: pandora)...
Create new df grouped by author...
	Create time columns...


  0%|          | 0/1057787 [00:00<?, ?it/s]

	Create new ungrouped columns...
	Group df by author...
Create new columns with features...
	Create controversiality column...
	Create mean_gilded...
	Create subreddit_dist...
	Compute time intervals...
	Create weekday_dist...
	Create month_dist...
	Create year_dist...
	Create day_dist...
	Create new aggregated df...
	Sort new aggregated df...
	Drop duplicates in new aggregated df...
Distribute the weekday_dist to several columns...


  0%|          | 0/7 [00:00<?, ?it/s]

Distribute the month_dist to several columns


...

	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
LDA with onehundred topics: 
Start LDA...


  0%|          | 0/386 [00:00<?, ?it/s]

	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with 

	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
	Done with this row...
End time: 2021-04-09 13:06:58.714070
<class 'pandas.core.frame.DataFrame'>
Index: 386 entries, -BlitzN9ne to zymmaster
Columns: 13000 entries, ('text', 'body') to ('lda', 'ldahundred')
dtypes: float64(5102), int64(7891), object(7)
memory usage: 38.3+ MB
None


text  \
                                                           body   
author                                                            
-BlitzN9ne    I'm currently in the middle of making a Payday...   
-dyad-        I failed both...I'm great at reading people ir...   
12345jk12345  Are there any cheaper, decent quality black bo...   
64BitCoffee   Top 10 anime betrayals �� Always look on the b...   
ACE_C0ND0R    Contra? Your marching band plays G-bugles? Why...   

                                                                 \
                                                       doc_body   
author                                                            
-BlitzN9ne    [I'm currently in the middle of making a Payda...   
-dyad-        [I failed both...I'm great at reading people i...   
12345jk12345  [Are there any cheaper, decent quality black b...   
64BitCoffee   [Top 10 anime betrayals, �� Always look on the...   
ACE_C0ND0R    [Contra? Your marching band plays G-bugles?, W...   

                                                           data       post  \
                                                            utc      score   
author                                                                       
-BlitzN9ne    [1422166355, 1423504286, 1449881503, 145521567...   9.644956   
-dyad-        [1475875524, 1473096864, 1505168466, 150318014...   7.234043   
12345jk12345  [1515977214, 1525636082, 1515596601, 153118100...   2.054545   
64BitCoffee   [1526895742, 1518434252, 1525956068, 151835705...  10.816901   
ACE_C0ND0R    [1442261670, 1474081729, 1454349478, 143863353...   6.359872   

                                                           subtf            \
             controversiality    gilded   ratio_en num_subreddit   entropy   
author                                                                       
-BlitzN9ne           0.014159  0.000000   7.883648           116  4.865813   
-dyad-               0.000000  0.000000  32.571429             5  1.484707   
12345jk12345         0.018182  0.000000  17.333333            20  3.474704   
64BitCoffee          0.007042  0.000000   1.581818            14  1.537101   
ACE_C0ND0R           0.039165  0.000321  14.812183           301  5.604323   

                             ...  lin_feat                                \
                  mean_time  ... fp_plural   secondp   tp_sing tp_plural   
author                       ...                                           
-BlitzN9ne     48306.475567  ...  0.043540  0.425487  0.316106  0.138761   
-dyad-        379973.713675  ...  0.097872  0.272340  1.285106  0.659574   
12345jk12345  374470.440367  ...  0.063636  1.181818  0.845455  0.154545   
64BitCoffee   269992.624113  ...  0.007042  0.077465  0.119718  0.014085   
ACE_C0ND0R     21900.344518  ...  0.090851  0.369663  0.557464  0.210754   

                                          psych                lda             
             indef_pronouns aux_verbs mrc_cmean mrc_pmean ldafifty ldahundred  
author                                                                         
-BlitzN9ne         0.416991  1.282478       0.0       0.0       16         61  
-dyad-             1.557447  4.365957       0.0       0.0       23          1  
12345jk12345       1.109091  2.627273       0.0       0.0       18         32  
64BitCoffee        0.098592  0.274648       0.0       0.0       47         24  
ACE_C0ND0R         0.543499  1.769342       0.0       0.0        7         52  

[5 rows x 13000 columns]

### Naming
Big Five Labels: b5feat

Big Five Labels + Augmentation: b5feat_aug

Big Five + MBTI Labels: b5mbtifeat

Big Five + MBTI Labels + Augmentation: b5mbti_aug